In [1]:
import csv
import tweepy
import ssl
import twitter_credentials


consumer_key = twitter_credentials.CONSUMER_KEY 
consumer_secret = twitter_credentials.CONSUMER_SECRET
access_token = twitter_credentials.ACCESS_TOKEN
access_token_secret = twitter_credentials.ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
ssl._create_default_https_context = ssl._create_unverified_context
api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True)

user = api.me()
print (user.name)

Usha Chari


In [8]:
#Trump info to be scraped from Twitter site
name = 'realDonaldTrump'
tweet_id = '1311804792600227841'


In [9]:
#Scrape thru Twitters API
replies=[]
for tweet in tweepy.Cursor(api.search,q='to:'+name, result_type='recent', timeout=999999).items(1000):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            replies.append(tweet)

with open('twitter_trump_data.csv', 'a+') as f:
    csv_writer = csv.DictWriter(f, fieldnames=('user', 'text'))
    csv_writer.writeheader()
    for tweet in replies:
        row = {'user': tweet.user.screen_name, 'text': tweet.text.replace('\n', ' ')}
        csv_writer.writerow(row)

In [15]:
# Copy twitter info into CSV file
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

csv_file = "twitter_trump_data.csv"
twitter_trump_data_df = pd.read_csv(csv_file)
twitter_trump_data_df.head()

,user,text
0,TheWondersNatur,@realDonaldTrump Proud Boys Celebrate After Tr...
1,johnnyzom,@realDonaldTrump https://t.co/83FP4TgSBs.
2,annmill1966,@realDonaldTrump Get ready to go to prison Don...
3,WaynesWorld1960,@realDonaldTrump 20 times Trump disavowed whit...
4,2bconsidered,@realDonaldTrump https://t.co/CA3riZPh6z


In [7]:
#Biden info to be scraped from Twitter site
name = 'JoeBiden'
tweet_id = '1311807606365790210'

replies=[]
for tweet in tweepy.Cursor(api.search,q='to:'+name, result_type='recent', timeout=999999).items(1000):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            replies.append(tweet)

with open('twitter_biden_data.csv', 'a+') as f:
    csv_writer = csv.DictWriter(f, fieldnames=('user', 'text'))
    csv_writer.writeheader()
    for tweet in replies:
        row = {'user': tweet.user.screen_name, 'text': tweet.text.replace('\n', ' ')}
        csv_writer.writerow(row)

In [25]:
##Biden info to Excel file
csv_file = "twitter_biden_data.csv"
twitter_biden_data_df = pd.read_csv(csv_file)
twitter_biden_data_df.head()

,user,text
0,AlofsLisa,@JoeBiden What are your comments on this Joe B...
1,Me4ok1,"@JoeBiden America is ""laughing with you"" Presi..."
2,carenmess,@JoeBiden Shut up man
3,WeezieJean630,@JoeBiden Keep up the positive messaging! It’...
4,TylerB21495663,@JoeBiden Your one of the reasons the family u...


In [39]:
#Load the dataframes to Mongo DB using a new function

def write_df_to_mongoDB(  my_df,\
                          database_name = 'mydatabasename' ,\
                          collection_name = 'mycollectionname',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 100):
    #"""
    #This function take a list and create a collection in MongoDB (you should
    #provide the database name, collection, port to connect to the remoete database,
    #server of the remote database, local port to tunnel to the other machine)
    #
    #---------------------------------------------------------------------------
    #Parameters / Input
    #    my_list: the list to send to MongoDB
    #    database_name:  database name
    #
    #    collection_name: collection name (to create)
    #    server: the server of where the MongoDB database is hosted
    #        Example: server = '132.434.63.86'
    #    this_machine_port: local machine port.
    #        For example: this_machine_port = '27017'
    #    remote_port: the port where the database is operating
    #        For example: remote_port = '27017'
    #    chunk_size: The number of items of the list that will be send at the
    #        some time to the database. Default is 100.
    #
    #Output
    #    When finished will print "Done"
    #----------------------------------------------------------------------------
    #FUTURE modifications.
    #1. Write to SQL
    #2. Write to csv
    #----------------------------------------------------------------------------
    #30/11/2017: Rafael Valero-Fernandez. Documentation
    #"""

    #To connect
    import os
    import pandas as pd
    import pymongo
    from pymongo import MongoClient

    client = MongoClient('localhost',int(mongodb_port))
    db = client[database_name]
    collection = db[collection_name]
    # To write
    collection.delete_many({})  # Destroy the collection
    #aux_df=aux_df.drop_duplicates(subset=None, keep='last') # To avoid repetitions
    my_list = my_df.to_dict('records')
    l =  len(my_list)
    ran = range(l)
    #print(1,ran)
    steps=list(ran[chunk_size::chunk_size])
    steps.append([l])

    #steps=ran[chunk_size::chunk_size]
    #steps.extend([l])
    

    # Inser chunks of the dataframe
    i = 0
    for j in steps:
        #print(j,i)
        if type(j) != list:
            collection.insert_many(my_list[i:j]) # fill de collection
        i = j

    print('Done')
    return

In [40]:
write_df_to_mongoDB(twitter_biden_data_df,\
                          database_name = 'ElectionForecast2020' ,\
                          collection_name = 'twitter_biden_data',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 100)

Done


In [41]:
write_df_to_mongoDB(twitter_trump_data_df,\
                          database_name = 'ElectionForecast2020' ,\
                          collection_name = 'twitter_trump_data',
                          server = 'localhost',\
                          mongodb_port = 27017,\
                          chunk_size = 10)

Done
